In [12]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
data = pd.read_csv("./FL_data")

In [13]:
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import argparse
import seaborn as sns
import os
import hashlib
from transformers import pipeline
from sklearn.metrics import confusion_matrix
from PIL import Image
from sklearn.cluster import BisectingKMeans
import numpy as np

In [5]:
from PIL import Image
img = Image.open("./archive/imagenet-10/n02056570/n02056570_41.JPEG")

In [15]:


#Check clients number
clients_ = []
for names in data['Position'].unique():
    if 'Client' in names:
        clients_.append(names)


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pipe_224= pipeline(task="image-feature-extraction", model_name="vit_base_patch16_224", device=DEVICE, pool=True)

for client_no in clients_:
    target_data_csv = data[data['Position']==client_no]

    Folders = target_data_csv['Folder']
    Files = target_data_csv['File']
    img_list = []
    output_feature = []
    batch_size = 1000
    
    for i in range(0, len(Folders), batch_size):
        batch_folders = Folders[i:i+batch_size]
        batch_files = Files[i:i+batch_size]
        batch_imgs = []
        
        for Folder, File in zip(batch_folders, batch_files):
            path = os.path.join(Folder, File)
            path = os.path.join("./archive/imagenet-10", path)
            try:
                img = Image.open(path)
                batch_imgs.append(img)
            except Exception as e:
                print(f"Error opening image '{path}': {e}")
        
        batch_feature = pipe_224(batch_imgs)
        batch_feature = np.array(batch_feature)
        batch_feature_flat = batch_feature.reshape(batch_feature.shape[0], -1)
    
        output_feature.append(batch_feature_flat)
    
        # Close opened images
        for img in batch_imgs:
            img.close()

    output_feature = np.concatenate(output_feature, axis=0)
    print(f'Openning Successed!')
    if 'name' in target_data_csv.columns:
        label = target_data_csv['name']
    else:
        raise ValueError(f'No label on dataset[\'name\']')
    
    n_cluster = len(label.unique())
    
    # Feature Extracting
    KM = KMeans()
    BK_y = KM.fit_predict(output_feature)

    #Needs a fix
    names = ['Penguin','Dog','Cheetah','Plane','Zeppelin','Ship','SoccerBall','Car','Truck','Orange']
    cluster_to_name = {i: names[i] for i in range(n_cluster)}
    BK_y_named = [cluster_to_name[label] for label in BK_y]

    conf_matrix = confusion_matrix(label, BK_y_named)
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted Label')
    plt.ylabel('Class')

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 3f49326 (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\seung\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 